## Comparison to yfinance

In [1]:
import os

import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

from yahooquery import Ticker
import yfinance as yf

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,}'.format
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)

In [2]:
symbol = 'aapl'

YF_aapl = yf.Ticker('aapl')
YQ_aapl = Ticker('aapl')

### Financials

#### yfinance

In [3]:
%%time
attrs = ['cashflow', 'balance_sheet', 'financials']
yf_d = {}
for attr in attrs:
    df = getattr(YF_aapl, attr)
    yf_d[attr] = df.shape[0]

Wall time: 3.19 s


#### yahooquery

In [4]:
%%time
methods = ['cash_flow', 'balance_sheet', 'income_statement']
yq_d = {}
for method in methods:
    df = getattr(YQ_aapl, method)()
    df.set_index(['asOfDate', 'periodType'], inplace=True)
    df = df.T
    yq_d[method] = df.shape[0]

Wall time: 1.45 s


In [5]:
# Comparison of the number of items returned for each
cash_flow = yf_d.pop('cashflow')
income_statement = yf_d.pop('financials')
yf_d.update({'cash_flow': cash_flow, 'income_statement': income_statement})
pd.DataFrame({'yahooquery': yq_d, 'yfinance': yf_d})

yahooquery  yfinance
cash_flow                 26        19
balance_sheet             66        25
income_statement          39        22

Yahooquery retrieves more cash flow, balance sheet, and income statement data than yfinance and quicker!

### Historical Pricing

In [6]:
# Compare downloads for all companies within the S&P500
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500 = tables[0]['Symbol'].tolist()
sp500 = [symbol.replace(".", "-") for symbol in sp500]

#### yfinance

In [7]:
%%time
yf_data = yf.download(sp500, period='ytd', interval='1d', group_by='ticker')

[*********************100%***********************]  505 of 505 completed
Wall time: 42.5 s


In [8]:
type(yf_data)
yf_data.shape
yf_data.head()

pandas.core.frame.DataFrame

(94, 3030)

PSX                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02 112.08000183105469 113.18000030517578 111.37999725341797   
2020-01-03             111.25 112.41999816894531 108.19000244140625   
2020-01-06 108.23999786376953 108.44000244140625 107.20999908447266   
2020-01-07 107.12999725341797 108.22000122070312 106.73999786376953   
2020-01-08 108.31999969482422 108.44000244140625 103.80999755859375   

                                                                        VFC  \
                        Close          Adj Close   Volume              Open   
Date                                                                          
2020-01-02 112.19999694824219 109.68885040283203  1607900 99.66999816894531   
2020-01-03 108.48999786376953  106.0618896484375  2236900  98.9000015258789   
2020-01-06 107.80999755859375 105.39710998535156  3917100              99.0   
2020-01-07 108.13999938964844 105.71971893310547  2687900 99.16999816894531   
2020-01-08  104.0999984741211 101.77013397216797  3452600              98.0   

                                                                  \
                        High               Low             Close   
Date                                                               
2020-01-02            100.25  99.4800033569336 100.2300033569336   
2020-01-03 99.94000244140625 98.76000213623047  99.5999984741211   
2020-01-06 99.43000030517578  98.5999984741211 99.20999908447266   
2020-01-07 99.97000122070312 98.55999755859375 98.72000122070312   
2020-01-08 98.18000030517578 96.80999755859375 97.04000091552734   

                                                     IPG                     \
                   Adj Close   Volume               Open               High   
Date                                                                          
2020-01-02 99.54398345947266  2555100 23.270000457763672 23.360000610351562   
2020-01-03 98.91828918457031  1966200  22.40999984741211 22.780000686645508   
2020-01-06 98.53096008300781  1967600 22.579999923706055 22.950000762939453   
2020-01-07 98.04431915283203  1766100   22.8799991607666 23.020000457763672   
2020-01-08 96.37581634521484  2980000 23.020000457763672  23.15999984741211   

                                                                              \
                          Low              Close          Adj Close   Volume   
Date                                                                           
2020-01-02 22.389999389648438 22.709999084472656 22.444355010986328  4513100   
2020-01-03  22.40999984741211 22.729999542236328 22.464120864868164  2302000   
2020-01-06 22.520000457763672  22.93000030517578  22.66178321838379  3321100   
2020-01-07 22.760000228881836 22.979999542236328 22.711196899414062  2853400   
2020-01-08  22.81999969482422 23.040000915527344 22.770496368408203  4438200   

                         BIIB                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02 299.05999755859375  299.0799865722656  292.2799987792969   
2020-01-03  290.7699890136719 292.67999267578125  290.2200012207031   
2020-01-06   290.239990234375 291.19000244140625 288.20001220703125   
2020-01-07 290.57000732421875  291.8399963378906   287.989990234375   
2020-01-08  288.8800048828125 294.19000244140625 287.80999755859375   

                                                                        MNST  \
                        Close          Adj Close   Volume               Open   
Date                                                                           
2020-01-02   294.239990234375   294.239990234375  1178300 63.959999084472656   
2020-01-03  290.8500061035156  290.8500061035156   700100 63.119998931884766   
2020-01-06 290.82000732421875 290.82

#### yahooquery

In [9]:
%%time
tickers = Ticker(sp500, asynchronous=True)
yq_data = tickers.history(period='ytd', interval='1d')

Wall time: 12.3 s


In [10]:
type(yq_data)
yq_data.shape
yq_data.head()

pandas.core.frame.DataFrame

(47263, 8)

high              close   volume  \
symbol date                                                                 
MMM    2020-01-02 14:30:00 180.00999450683594              180.0  3601700   
       2020-01-03 14:30:00 178.66000366210938  178.4499969482422  2466900   
       2020-01-06 14:30:00  178.7100067138672  178.6199951171875  1998000   
       2020-01-07 14:30:00 178.50999450683594 177.89999389648438  2173000   
       2020-01-08 14:30:00              181.5  180.6300048828125  2758300   

                                         open                low  \
symbol date                                                        
MMM    2020-01-02 14:30:00 177.67999267578125 177.13999938964844   
       2020-01-03 14:30:00 177.02000427246094  175.6300048828125   
       2020-01-06 14:30:00 177.14999389648438 176.35000610351562   
       2020-01-07 14:30:00 178.27999877929688 176.82000732421875   
       2020-01-08 14:30:00              178.0 177.64999389648438   

                                     adjclose  dividends  splits  
symbol date                                                       
MMM    2020-01-02 14:30:00 178.38461303710938        0.0     0.0  
       2020-01-03 14:30:00 176.84852600097656        0.0     0.0  
       2020-01-06 14:30:00 177.01699829101562        0.0     0.0  
       2020-01-07 14:30:00 176.30345153808594        0.0     0.0  
       2020-01-08 14:30:00  179.0089569091797        0.0     0.0

### Option Chain

#### yfinance

In [19]:
%%time
dataframes = []
for option in YF_aapl.options:
    options = YF_aapl.option_chain(option)
    calls = options[0]
    calls['optionType'] = 'calls'
    puts = options[1]
    puts['optionType'] = 'puts'
    dataframes.append(pd.concat([calls, puts]))
yf_option_chain = pd.concat(dataframes)

Wall time: 3.49 s


In [13]:
yf_option_chain

contractSymbol       lastTradeDate  strike  lastPrice    bid    ask  \
0   AAPL200522C00150000 2020-05-07 19:54:28   150.0     153.03 157.05 158.45   
1   AAPL200522C00160000 2020-04-20 19:53:12   160.0     118.07 147.05 148.45   
2   AAPL200522C00165000 2020-05-07 19:25:55   165.0     138.05 142.05 143.45   
3   AAPL200522C00170000 2020-05-13 23:08:44   170.0      95.65  132.4 133.65   
4   AAPL200522C00190000 2020-04-22 13:39:27   190.0      84.64 117.05 118.45   
..                  ...                 ...     ...        ...    ...    ...   
43  AAPL220617P00450000 2020-03-16 19:24:29   450.0     207.11 172.15 176.65   
44  AAPL220617P00460000 2020-05-13 16:17:30   460.0      168.4  164.5  168.0   
45  AAPL220617P00470000 2020-05-11 17:58:15   470.0     169.15  173.2  176.4   
46  AAPL220617P00480000 2020-05-08 16:25:27   480.0     183.77  182.0  185.2   
47  AAPL220617P00485000 2020-03-26 16:04:30   485.0      236.0  206.5  209.8   

    change  percentChange  volume  openInterest      impliedVolatility  \
0      0.0            0.0     nan           0.0        1.9921875390625   
1      0.0            0.0     2.0           0.0     1.8281258593749998   
2      0.0            0.0     2.0           0.0     1.7421887890624999   
3      0.0            0.0     nan           0.0 1.0000000000000003e-05   
4      0.0            0.0     nan           0.0     1.3750031249999999   
..     ...            ...     ...           ...                    ...   
43     0.0            0.0     2.0           2.0     0.4131833584594726   
44     0.0            0.0     1.0           1.0     0.3032601412963867   
45     0.0            0.0     1.0          15.0    0.30049832809448246   
46     0.0            0.0     1.0           1.0     0.2998727239990234   
47     0.0            0.0     6.0           7.0    0.43629256340026856   

    inTheMoney contractSize currency optionType  
0         True      REGULAR      USD      calls  
1         True      REGULAR      USD      calls  
2         True      REGULAR      USD      calls  
3         True      REGULAR      USD      calls  
4         True      REGULAR      USD      calls  
..         ...          ...      ...        ...  
43        True      REGULAR      USD       puts  
44        True      REGULAR      USD       puts  
45        True      REGULAR      USD       puts  
46        True      REGULAR      USD       puts  
47        True      REGULAR      USD       puts  

[1760 rows x 15 columns]

#### yahooquery

In [17]:
%%time
yq_option_chain = YQ_aapl.option_chain

Wall time: 224 ms


In [18]:
yq_option_chain

ask    bid  change contractSize  \
symbol expiration optionType                                      
aapl   2020-05-22 calls      158.45 157.05     0.0      REGULAR   
                  calls      148.45 147.05     0.0      REGULAR   
                  calls      143.45 142.05     0.0      REGULAR   
                  calls      133.65  132.4     0.0      REGULAR   
                  calls      118.45 117.05     0.0      REGULAR   
...                             ...    ...     ...          ...   
       2022-06-17 puts       176.65 172.15     0.0      REGULAR   
                  puts        168.0  164.5     0.0      REGULAR   
                  puts        176.4  173.2     0.0      REGULAR   
                  puts        185.2  182.0     0.0      REGULAR   
                  puts        209.8  206.5     0.0      REGULAR   

                                   contractSymbol currency  \
symbol expiration optionType                                 
aapl   2020-05-22 calls       AAPL200522C00150000      USD   
                  calls       AAPL200522C00160000      USD   
                  calls       AAPL200522C00165000      USD   
                  calls       AAPL200522C00170000      USD   
                  calls       AAPL200522C00190000      USD   
...                                           ...      ...   
       2022-06-17 puts        AAPL220617P00450000      USD   
                  puts        AAPL220617P00460000      USD   
                  puts        AAPL220617P00470000      USD   
                  puts        AAPL220617P00480000      USD   
                  puts        AAPL220617P00485000      USD   

                                  impliedVolatility  inTheMoney  lastPrice  \
symbol expiration optionType                                                 
aapl   2020-05-22 calls             1.9921875390625        True     153.03   
                  calls          1.8281258593749998        True     118.07   
                  calls          1.7421887890624999        True     138.05   
                  calls      1.0000000000000003e-05        True      95.65   
                  calls          1.3750031249999999        True      84.64   
...                                             ...         ...        ...   
       2022-06-17 puts           0.4131833584594726        True     207.11   
                  puts           0.3032601412963867        True      168.4   
                  puts          0.30049832809448246        True     169.15   
                  puts           0.2998727239990234        True     183.77   
                  puts          0.43629256340026856        True      236.0   

                                   lastTradeDate  openInterest  percentChange  \
symbol expiration optionType                                                    
aapl   2020-05-22 calls      2020-05-07 19:54:28           0.0            0.0   
                  calls      2020-04-20 19:53:12           0.0            0.0   
                  calls      2020-05-07 19:25:55           0.0            0.0   
                  calls      2020-05-13 23:08:44           0.0            0.0   
                  calls      2020-04-22 13:39:27           0.0            0.0   
...                                          ...           ...            ...   
       2022-06-17 puts       2020-03-16 19:24:29           2.0            0.0   
                  puts       2020-05-13 16:17:30           1.0            0.0   
                  puts       2020-05-11 17:58:15          15.0            0.0   
                  puts       2020-05-08 16:25:27           1.0            0.0   
                  puts       2020-03-26 16:04:30           7.0            0.0   

                              strike  volume  
symbol expiration optionType                  
aapl   2020-05-22 calls        150.0     0.0  
                  calls        160.0     2.0  
                  calls        165.0     2.0  
                  calls        170.0     0.0  
  

Retrieving all option expiration dates is a simple one-liner in yahooquery (and faster)!

### Multiple Symbols

In [20]:
symbols = 'FB AAPL AMZN NFLX GOOG'

#### yfinance

In [21]:
%%time
attrs = [
    'info', 'financials', 'quarterly_financials', 'major_holders',
    'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet',
    'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings',
    'sustainability', 'recommendations', 'calendar'
]
yf_d = {}
tickers = yf.Tickers(symbols)
for symbol in symbols.split(" "):
    ticker = getattr(tickers.tickers, symbol)
    yf_d[symbol] = {}
    for attr in attrs:
        yf_d[symbol][attr] = getattr(ticker, attr)

Wall time: 12.8 s


In [26]:
yf_d.keys()
yf_d['FB'].keys()

dict_keys(['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG'])

dict_keys(['info', 'financials', 'quarterly_financials', 'major_holders', 'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet', 'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings', 'sustainability', 'recommendations', 'calendar'])

#### yahooquery

In [23]:
%%time
tickers = Ticker(symbols, asynchronous=True)
data = tickers.all_modules

Wall time: 436 ms


In [25]:
data.keys()
data['FB'].keys()

dict_keys(['AMZN', 'AAPL', 'NFLX', 'GOOG', 'FB'])

dict_keys(['assetProfile', 'recommendationTrend', 'cashflowStatementHistory', 'indexTrend', 'defaultKeyStatistics', 'industryTrend', 'quoteType', 'incomeStatementHistory', 'fundOwnership', 'summaryDetail', 'insiderHolders', 'calendarEvents', 'upgradeDowngradeHistory', 'price', 'balanceSheetHistory', 'earningsTrend', 'secFilings', 'institutionOwnership', 'majorHoldersBreakdown', 'balanceSheetHistoryQuarterly', 'earningsHistory', 'esgScores', 'summaryProfile', 'netSharePurchaseActivity', 'insiderTransactions', 'sectorTrend', 'incomeStatementHistoryQuarterly', 'cashflowStatementHistoryQuarterly', 'earnings', 'pageViews', 'financialData'])

Again, retrieving data, this time with multiple symbols, is faster and simpler!